In [1]:
import geopandas as gpd
from pathlib import Path
import matplotlib.pyplot as plt
import contextily as cx
import fiona 
import numpy as np
# Potentiel solaire package
from potentiel_solaire.constants import DATA_FOLDER

In [ ]:
# Executer ci dessous ci besoin pour récupérer les données
# !extract-sample-data
# Et pour sauver une version markdown des notebooks, utiliser
# jupyter nbconvert donnees_par_ecole.ipynb --to markdown --output-dir=exports/

In [ ]:
# Verification des layers disponibles
stdenis_path = DATA_FOLDER / "saint_denis_reference_data.gpkg"
layers = fiona.listlayers(stdenis_path)
print(" ".join(layers))

## On ouvre le jeu de données préparé

In [3]:
# On ouvre une zone
ID = 6
stdenis = gpd.read_file(stdenis_path, layer="bdtopo_education")# toponymie_services_et_activites
stdenis = stdenis.to_crs(2154) # 2154 Lambert, 4326 latlon  
example = stdenis[ID:ID+1]
ID = example["cleabs_left"].iloc[0]
NOM = example["toponyme"].iloc[0]
# On ouvre les toitures
toiture = gpd.read_file(stdenis_path, layer="bdtopo_batiment")# toponymie_services_et_activites
toiture_b = gpd.read_file(stdenis_path, layer="potentielsolaire_bati")# toponymie_services_et_activites
toiture_t = gpd.read_file(stdenis_path, layer="potentielsolaire_toitures")# toponymie_services_et_activites
toiture_c = gpd.read_file(stdenis_path, layer="cadastre_parcellaire")# toponymie_services_et_activites

toiture = toiture.to_crs(2154) # 2154 Lambert, 4326 latlon  
toiture_b = toiture.to_crs(2154)
toiture_t = toiture.to_crs(2154)
toiture_c = toiture.to_crs(2154)
toiture = gpd.sjoin(toiture, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri').clip(example)
toiture_b = gpd.sjoin(toiture_b, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri').clip(example)
toiture_t = gpd.sjoin(toiture_t, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri').clip(example)
toiture_c = gpd.sjoin(toiture_c, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri').clip(example)

In [ ]:
fig, ax = plt.subplots()

example.plot(ax=ax, alpha=0.7, color ="green")
toiture_c.plot(ax=ax, alpha=0.7, color ="pink")
toiture_t.plot(ax=ax, alpha=0.7, color ="yellow")
toiture_b.plot(ax=ax, alpha=0.7, color ="orange")
toiture.plot(ax=ax, alpha=0.7, color ="red")


cx.add_basemap(ax, crs=example.crs)
fig.show()

# Exploration MNS liées aux zones

In [5]:
import rasterio
import rasterio.mask
import os
from rasterio.plot import show

## On recadre l'image

In [6]:
geotiff_cached = "../data/cache/mns/"+ID+".masked.tif"
os.makedirs("../data/cache/mns", exist_ok=True)

if not os.path.isfile(geotiff_cached):
    geome = example.geometry.total_bounds
    A = "0"+str(int(geome[0]//1000))
    B = str(int(geome[1]//1000)+1)
    tile = "/MNS-Correl_1-0__TIFF_LAMB93_D093_2024-01-01/MNS-Correl/1_DONNEES_LIVRAISON_2024-11-00179/MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-"+A+"-"+B+"-LA93-0M50.tif"
    path = DATA_FOLDER / tile
    with rasterio.open("../data/"+str(path)) as img:
        out_image, out_transform = rasterio.mask.mask(img, example.geometry, crop=True)
        out_meta = img.meta
    minval = np.min(out_image[np.nonzero(out_image)])
    out_image = out_image - minval

    out_image = np.where(out_image<0, 0, out_image)
    out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})

    with rasterio.open("../data/cache/mns/"+ID+".masked.tif", "w", **out_meta) as dest:
        dest.write(out_image)

## Et on la représente

In [ ]:
img_ecole = rasterio.open("../data/cache/mns/"+ID+".masked.tif")

fig, ax = plt.subplots(figsize=(10,10))



show(img_ecole, ax=ax, alpha=0.9)
example.plot(ax=ax, alpha=0.7, linewidth=2, facecolor="none", edgecolor='pink')
toiture_c.plot(ax=ax, alpha=0.7, linewidth=3,facecolor="none", edgecolor='green')
toiture.plot(ax=ax, alpha=0.7, linewidth=4,facecolor="none", edgecolor='red', label="batiments")
toiture_t.plot(ax=ax, alpha=0.7, linewidth=2,facecolor="none", edgecolor='orange')
toiture_b.plot(ax=ax, alpha=0.7, linewidth=3,facecolor="none", edgecolor='yellow')


cx.add_basemap(ax, crs=example.crs, alpha=0.5, source=cx.providers.GeoportailFrance.orthos )


plt.title("Exploitation MNS pour :"+ NOM+"\n"+ID+"\n")
fig.show()



# Focalisation sur les batiments

In [8]:
geotiff_cached_toits = "../data/cache/mns/"+ID+"_toits.masked.tif"

if not os.path.isfile(geotiff_cached_toits) or True:
    geome = example.geometry.total_bounds
    A = "0"+str(int(geome[0]//1000))
    B = str(int(geome[1]//1000)+1)
    tile = "/MNS-Correl_1-0__TIFF_LAMB93_D093_2024-01-01/MNS-Correl/1_DONNEES_LIVRAISON_2024-11-00179/MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-"+A+"-"+B+"-LA93-0M50.tif"
    path = DATA_FOLDER / tile
    with rasterio.open("../data/"+str(path)) as img:
        out_image, out_transform = rasterio.mask.mask(img, toiture.geometry, crop=True)
        out_meta = img.meta
    minval = np.min(out_image[np.nonzero(out_image)])
    out_image = out_image - minval

    out_image = np.where(out_image<0, 0, out_image)

    MAX = np.percentile(out_image[np.nonzero(out_image)],80)
    out_image = np.where(out_image>=MAX, MAX, out_image)

    out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})

    with rasterio.open(geotiff_cached_toits, "w", **out_meta) as dest:
        dest.write(out_image)

In [ ]:
img_ecole = rasterio.open(geotiff_cached_toits)

fig, ax = plt.subplots(figsize=(10,10))

show(img_ecole, ax=ax, alpha=0.8)

example.plot(ax=ax, alpha=0.7, linewidth=2, facecolor="none", edgecolor='pink')
# A explorer peut etre: cx.providers.GeoportailFrance
cx.add_basemap(ax, crs=example.crs, alpha=0.6, source=cx.providers.GeoportailFrance.orthos )
toiture_c.plot(ax=ax, alpha=0.7, linewidth=3,facecolor="none", edgecolor='green')
toiture.plot(ax=ax, alpha=0.7, linewidth=4,facecolor="none", edgecolor='red')
toiture_t.plot(ax=ax, alpha=0.7, linewidth=2,facecolor="none", edgecolor='orange')
toiture_b.plot(ax=ax, alpha=0.7, linewidth=3,facecolor="none", edgecolor='yellow')


plt.title("Exploitation MNS pour :"+ NOM+"\n"+ID+"\n")
fig.show()

# Et sur toutes les écoles

In [10]:
stdenis = gpd.read_file(stdenis_path, layer="bdtopo_education")# toponymie_services_et_activites
stdenis = stdenis.to_crs(2154) # 2154 Lambert, 4326 latlon  

In [11]:
toiturefull = gpd.read_file(stdenis_path, layer="bdtopo_batiment")# toponymie_services_et_activites
toiture_bfull = gpd.read_file(stdenis_path, layer="potentielsolaire_bati")# toponymie_services_et_activites
toiture_tfull = gpd.read_file(stdenis_path, layer="potentielsolaire_toitures")# toponymie_services_et_activites
toiture_cfull = gpd.read_file(stdenis_path, layer="cadastre_parcellaire")# toponymie_services_et_activites
toiturefull = toiturefull.to_crs(2154) # 2154 Lambert, 4326 latlon  
toiture_bfull = toiture_bfull.to_crs(2154)
toiture_tfull = toiture_tfull.to_crs(2154)
toiture_cfull = toiture_cfull.to_crs(2154)

In [ ]:
OnVeutRefaireTousLesToits = True
if OnVeutRefaireTousLesToits:
    for ID in range(len(stdenis)-1):
        example = stdenis[ID:ID+1]
        ID = example["cleabs_left"].iloc[0]
        NOM = example["toponyme"].iloc[0]
        print(ID)

        toiture = gpd.sjoin(toiturefull, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri')
        toiture_b = gpd.sjoin(toiture_bfull, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri')
        toiture_t = gpd.sjoin(toiture_tfull, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri')
        toiture_c = gpd.sjoin(toiture_cfull, example, how='inner', predicate='intersects',lsuffix='_li', rsuffix='_ri')
        geotiff_cached = "../data/cache/mns/"+ID+".masked.tif"
        try:

            if not os.path.isfile(geotiff_cached):
                geome = example.geometry.total_bounds
                A = "0"+str(int(geome[0]//1000))
                B = str(int(geome[1]//1000)+1)
                tile = "/MNS-Correl_1-0__TIFF_LAMB93_D093_2024-01-01/MNS-Correl/1_DONNEES_LIVRAISON_2024-11-00179/MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-"+A+"-"+B+"-LA93-0M50.tif"
                path = DATA_FOLDER / tile
                with rasterio.open("../data/"+str(path)) as img:
                    out_image, out_transform = rasterio.mask.mask(img, example.geometry, crop=True)
                    out_meta = img.meta
                minval = np.min(out_image[np.nonzero(out_image)])
                out_image = out_image - minval

                out_image = np.where(out_image<0, 0, out_image)
                out_meta.update({"driver": "GTiff",
                                "height": out_image.shape[1],
                                "width": out_image.shape[2],
                                "transform": out_transform})

                with rasterio.open("../data/cache/mns/"+ID+".masked.tif", "w", **out_meta) as dest:
                    dest.write(out_image)


            geotiff_cached_toits = "../data/cache/mns/"+ID+"_toits.masked.tif"

            if not os.path.isfile(geotiff_cached_toits) or True:
                geome = example.geometry.total_bounds
                A = "0"+str(int(geome[0]//1000))
                B = str(int(geome[1]//1000)+1)
                tile = "/MNS-Correl_1-0__TIFF_LAMB93_D093_2024-01-01/MNS-Correl/1_DONNEES_LIVRAISON_2024-11-00179/MNS-C_0M50_TIF_LAMB93_D93-2024/93-2024-"+A+"-"+B+"-LA93-0M50.tif"
                path = DATA_FOLDER / tile
                with rasterio.open("../data/"+str(path)) as img:
                    out_image, out_transform = rasterio.mask.mask(img, toiture.geometry, crop=True)
                    out_meta = img.meta
                minval = np.min(out_image[np.nonzero(out_image)])
                out_image = out_image - minval

                out_image = np.where(out_image<0, 0, out_image)

                MAX = np.percentile(out_image[np.nonzero(out_image)],80)
                out_image = np.where(out_image>=MAX, MAX, out_image)

                out_meta.update({"driver": "GTiff",
                                "height": out_image.shape[1],
                                "width": out_image.shape[2],
                                "transform": out_transform})

                with rasterio.open(geotiff_cached_toits, "w", **out_meta) as dest:
                    dest.write(out_image)


            img_ecole = rasterio.open(geotiff_cached_toits)

            fig, ax = plt.subplots(figsize=(10,10))

            show(img_ecole, ax=ax, alpha=0.8)

            example.plot(ax=ax, alpha=0.7, linewidth=2, facecolor="none", edgecolor='pink')
            # A explorer peut etre: cx.providers.GeoportailFrance 
            cx.add_basemap(ax, crs=example.crs, alpha=0.6, source=cx.providers.GeoportailFrance.orthos )
            toiture_c.plot(ax=ax, alpha=0.7, linewidth=3,facecolor="none", edgecolor='green')
            toiture.plot(ax=ax, alpha=0.7, linewidth=4,facecolor="none", edgecolor='red')
            toiture_t.plot(ax=ax, alpha=0.7, linewidth=2,facecolor="none", edgecolor='orange')
            toiture_b.plot(ax=ax, alpha=0.7, linewidth=3,facecolor="none", edgecolor='yellow')


            plt.title("Exploitation MNS pour :"+ NOM+"\n"+ID+"\n")
            os.makedirs("../data/cache/mns/jpg", exist_ok=True)
            plt.savefig("../data/cache/mns/jpg/"+ID+".jpg", bbox_inches='tight')
            #fig.show()
        except:
            print("Error with",ID)